In [3]:
import psycopg2
import pandas as pd
import statistics 
import random
from statistics import mode
from sklearn import metrics

#Establishing a connection with the Database:
conn = psycopg2.connect(
   database="DataMining", user= User,password = Password, host='localhost', port= '5432'
)

#Creating a cursor object using the cursor() method:
cursor = conn.cursor()


#Get list of movie genres of each user:
cursor.execute("SELECT userid,movieid,genres FROM user_movie_genres")
user_movie_genres = cursor.fetchall()
user_movie_gen = pd.DataFrame(user_movie_genres,columns= ['user','movie','genres'])
#print(user_movie_gen)
df = user_movie_gen.groupby('user')['genres'].apply(list).reset_index(name='genres_list')
#print(df)

#Extract mode of each list of movie genres:
num_of_lists = len(df)

lists = [[]]*num_of_lists
for i in range(num_of_lists):
    lists[i] = df['genres_list'].values[i]
    
modes = [None] *num_of_lists
for i in range(num_of_lists):
    modes[i] = mode(lists[i])
    
#Expand the dataframe containing users,movie genres of each movie each user has seen
#in order to include the favorite genre of each user.Delete non useful info:
df['Favorite_Genre'] = modes
del df["genres_list"]
del df["user"]

#print(df)

#Expand dataframe in order to include the age,gender and occupation of each user:
getAge = "SELECT age FROM users ORDER BY userid ASC"
getGender = "SELECT gender FROM users ORDER BY userid ASC"
getOccupation = "SELECT occupation FROM users ORDER BY userid ASC"


cursor.execute(getAge)
cursor_res1 = cursor.fetchall()
Age = [items[0] for items in cursor_res1]

cursor.execute(getGender)
cursor_res2 = cursor.fetchall()
Gender = [items[0] for items in cursor_res2]


cursor.execute(getOccupation)
cursor_res3 = cursor.fetchall()
Occupation = [items[0] for items in cursor_res3]

df['Age'] = Age
df['Gender'] = Gender
df['Occupation'] = Occupation

#print(df)

#Encode the Favorite_Genre column and add it to the dataframe
FavoriteGenres = df.Favorite_Genre.unique()
num_of_favorite_genres = len(FavoriteGenres)
DecimalGenres = [0] * num_of_favorite_genres

for i in range(num_of_favorite_genres):
    s = FavoriteGenres[i]
    DecimalGenres[i] = int(s, base=2)

FavoriteGenresDecimals = [None]*num_of_favorite_genres
for i in range(num_of_favorite_genres):
    FavoriteGenresDecimals[i] = str(DecimalGenres[i])
    

lastColumn = [None]*len(df)
for i in range(len(df)):
    for j in range(num_of_favorite_genres):
        if(df.iloc[i]['Favorite_Genre'] == FavoriteGenres[j] ):
            s = FavoriteGenresDecimals[j]
            break
    lastColumn[i] = s
    
del df["Favorite_Genre"]
df['Favorite_Genre'] = lastColumn

#print(df)
#Export df to a .csv file, in order to upload to Weka and extract decision tree
df.to_csv('/home/loutas/Documents/DATA MINING/Project/my_dataframe.csv',index = False,sep=',',header = True)


#Closing the connection
conn.close()
df

,Age,Gender,Occupation,Favorite_Genre
0,53,F,other,1024
1,23,M,writer,1024
2,24,M,technician,4100
3,33,F,other,8192
4,42,M,executive,1024
...,...,...,...,...
937,26,F,student,8192
938,32,M,administrator,1024
939,20,M,student,8192
940,48,F,librarian,1024


In [4]:
def FindGenre(age,gender,occupation):
    if(occupation == 'lawyer'):
        return '1040'
    
    if(occupation == 'administrator'):
        if(gender == 'F'):
            return '1040'
        if(gender == 'M'):
            if(age <= 35):
                if(age <= 29):
                    return '128'
                if(age > 29):
                    if(age <= 31):
                        return '196616'
                    if(age > 31):
                        return '128'
            if(age > 35):
                if(age <= 46):
                    return '131076'
                if(age > 46):
                    return '8208'
                       
    if(occupation == 'student'):
        if(gender == 'F'):
            if(age <= 21):
                if(age <= 19):
                    if(age <= 17):
                        return '8208'
                    if(age > 17):
                        return '1040'
                if(age > 19):
                    return '196612'
            if(age > 21):
                return '1040'
        if(gender == 'M'):
            if(age <= 21):
                if(age <= 13):
                    return '49216'
                if(age > 13):
                    if(age <= 18):
                        return '128'
                    if(age > 18):
                        return '8192'
            if(age > 21):
                if(age <= 24):
                    return '132'
                if(age > 24):
                    if(age <= 26):
                        if(age <= 25):
                            return '196616'
                        if(age > 25):
                            return '196634'
                    if(age > 26):
                        if(age <= 30):
                            return '8192'
                        if(age > 30):
                            return '8208'
                
    if(occupation == 'librarian'):
        if(gender == 'M'):
            return '128'
        if(gender == 'F'):
            return '1024'
    
    if(occupation == 'scientist'):
        if(age <= 43):
            if(age <= 41):
                if(age <= 38):
                    return '196616'
                if(age > 38):
                    return '8208'
            if(age > 41):
                return '196634'
        if(age > 43):
            return '132098'
        
    if(occupation == 'doctor'):
        return '131076'
    
    if(occupation == 'salesman'):
        return '128'
    
    if(occupation == 'retired'):
        if(age > 62):
            return '196612'
        if(age <= 62):
            return '131076'
        
    if(occupation == 'healthcare'):
        if(age > 44):
            return '131076'
        if(age <= 44):
            return '196616'
        
    if(occupation == 'none'):
        if(age > 31):
            return '196616'
        if(age <= 31):
            return '49216'
    
    if(occupation == 'marketing'):
        if(age > 34):
            return '1040'
        if(age <= 34):
            return '131076'
        
    if(occupation == 'engineer'):
        if(age <= 37):
            if(age <= 32):
                if(age <= 26):
                    return '8192'
                if(age > 26):
                    return '49216'
            if(age > 32):
                return '1026'
        if(age > 37):
            if(age <= 49):
                return '196616'
            if(age > 49):
                if(age <= 55):
                    return '24576'
                if(age > 55):
                    if(age <= 59):
                        return '1028'
                    if(age > 59):
                        return '1040'
                    
                
    if(occupation == 'artist'):
        if(gender == 'F'):
            return '1040'
        if(gender == 'M'):
            return '196616'
        
    if(occupation == 'homemaker'):
        if(age <= 21):
            return '24576'
        if(age > 21):
            return '9216'
    if(occupation == 'technician'):
        if(age <= 29):
            if(age <= 23):
                return '131076'
            if(age > 23):
                return '4100'
        if(age > 29):
            if(age <= 38):
                return '131084'
            if(age > 38):
                if(age <= 45):
                    return '1028'
                if(age > 45):
                    return '196616'
                
    if(occupation == 'entertainment'):
        if(age <= 20):
            return '4'
        if(age > 20):
            return '128'
        
    if(occupation == 'executive'):
        if(age <= 28):
            return '196612'
        if(age > 28):
            if(age <= 40):
                if(age <= 34):
                    return '8208'
                if(age > 34):
                    return '128'
            if(age > 40):
                return '1040'
            
    if(occupation == 'programmer'):
        if(gender == 'F'):
            if(age <= 31):
                return '139264'
            if(age > 31):
                return '196608'
        if(gender == 'M'):
            if(age <= 28):
                if(age <=23):
                    return '196616'
                if(age > 23):
                    return '1032'
            if(age > 28):
                if(age <= 34):
                    if(age <= 32):
                        return '8208'
                    if(age > 32):
                        return '131072'
                if(age > 34):
                    return '131076'
    
        
    if(occupation == 'other'):
        if(age <= 21):
            if(gender == 'F'):
                return '24640'
            if(gender == 'M'):
                return '131076'
        if(age > 21):
            if(age <= 34):
                if(age <=30):
                    if(age <= 27):
                        if(age <= 23):
                            return '131076'
                        if(age > 23):
                            return '8192'
                    if(age > 27):
                        return '1040'
                if(age > 30):
                    return '24576'
            if(age > 34):
                return '8208'
        
    
    if(occupation == 'writer'):
        if(age <= 23):
            if(age <= 20):
                return '49216'
            if(age > 20):
                return '132'
        if(age > 23):
            if(age <= 40):
                return '128'
            if(age > 40):
                return '196634'
            
    if(occupation == 'educator'):
        if(age <= 43):
            if(age <= 40):
                if(age <= 33):
                    if(age <= 29):
                        return '8208'
                    if(age > 29):
                        return '1040'
                if(age > 33):
                    if(age <= 36):
                        return '196616'
                    if(age > 36):
                        return '128'
            if(age > 40):
                return '196616'
        if(age > 43):
            if(age <= 48):
                if(age <= 45):
                    return '1040'
                if(age > 45):
                    return '24576'
            if(age > 48):
                if(age > 51):
                    return '1024'
                if(age <= 51):
                    if(gender == 'F'):
                        return '8208'
                    if(gender =='M'):
                        return '131076'

In [ ]:
#Get occupation list
Occupation_list = df.Occupation.unique()

#Reconnect with the database
conn = psycopg2.connect(
   database="DataMining", user='nloutas',password = 'Wh1teFang_98', host='localhost', port= '5432'
)

#Creating a cursor object using the cursor() method:
cursor = conn.cursor()

#Welcome user and make question
print('Welcome to the application!\nAre you a new user? ("Yes","No")\n')
newUser = input()

if(newUser == 'No'):
    print( 'Please enter your id:\n')
    id = input()
    
    print('Would you like to add some movies ("Yes","No") ?\n')
    addMovies = input()
    
    if(addMovies == 'Yes'):
        print("Give us some titles\n")
        
        #Give some movies and add them to the database
        while(1):
            new_movie = input()
            if(new_movie == 'exit'):
                break
           
            selectMovieElements = 'SELECT DISTINCT movieid, genres FROM movies ' 
            selectMovieElements = selectMovieElements + "WHERE moviename = '" + new_movie + "' "
            #print(selectMovieElements)
        
            cursor.execute(selectMovieElements)
            result = cursor.fetchall()
            
            if(len(result) == 0):
                print('Sorry,that movie is not included in our database :(')
            
            #movieId = result[0][0]
            #movieGenres = result[0][1]
            #insertNewMatch = 'insert into user_movie_genres(userid,movieid,genres) values('
            #insertNewMatch = insertNewMatch + id + "," + str(movieId) + "," + movieGenres + ')'
            #cursor.execute(insertNewMatch)
            #conn.commit()
    

    #Get some movies based on the tree
    getUserInfo = "SELECT age,gender,occupation FROM users"
    getUserInfo = getUserInfo + " WHERE userid = " + id
    cursor.execute(getUserInfo)
    info = cursor.fetchall()
        
    age = info[0][0]
    print(age)
    gender = info[0][1][0]
    gender = gender.replace(" ","")
    print(gender)
    occupation = info[0][2]
    occupation = occupation.replace(" ", "")
    print(occupation)
    #Find favorite genre:
    usersGenre = FindGenre(age,gender,occupation) #usersGenre is string
    #print(usersGenre)
    for i in range(len(FavoriteGenresDecimals)):
        if(usersGenre == FavoriteGenresDecimals[i] ):
            usersGenre = FavoriteGenres[i]
            break;
   
    #Get movies
    getMoviesFromTree = 'SELECT DISTINCT moviename FROM movies WHERE genres = ' + "'" + str(usersGenre) + "'"
    cursor.execute(getMoviesFromTree)
    moviesList1 = cursor.fetchall()

    #Get some movies based on the mode
    getMode = " SELECT genres, count(*) FROM user_movie_genres  "
    getMode = getMode + "WHERE userid = " + id
    getMode = getMode + " GROUP BY genres order by count(*) desc limit(1)"
    cursor.execute(getMode)
    mode = cursor.fetchall()
        
    mode = mode[0][0]

    getMoviesFromMode = "SELECT DISTINCT moviename FROM movies WHERE genres = "+ "'" + mode + "'"
    cursor.execute(getMoviesFromMode)
    moviesList2 = cursor.fetchall()
    
    
    FinalList = moviesList1 + moviesList2
    FinalList = set(FinalList)
    FinalList = list(FinalList)
        
    print("____________Here are some movies you may like______________________\n")
    if(len(FinalList) >= 10):
        FinalList = random.sample(FinalList,10)
        for i in range(10):
            print(FinalList[i][0])
    else:
        for i in range(len(FinalList)):
            print(FinalList[i][0])
        
if(newUser == 'Yes'):
    print('Please enter your age:\n')
    age = input()
    age = int(age)
    
    print('\nPlease enter your gender ("F","M"):\n')
    gender = input()
    gender = gender.replace(" ","")

    msg3 = '\nPlease enter your occupation from the list below:\n'
    newLine = '____________________________________________________'
    print(msg3 + newLine)
    
    for i in range(len(Occupation_list)):
        print(Occupation_list[i]+'\n')
    occupation = input()
    occupation = occupation.replace(" ", "")
    
    #Find favorite genre:
    usersGenre = FindGenre(age,gender,occupation) #usersGenre is string
    #print(usersGenre)
    for i in range(len(FavoriteGenresDecimals)):
        if(usersGenre == FavoriteGenresDecimals[i] ):
            usersGenre = FavoriteGenres[i]
            break;
    print(usersGenre)
   
    #Get movies
    getMoviesQuery = 'SELECT DISTINCT moviename FROM movies WHERE genres = ' + "'" + str(usersGenre) + "'"
    cursor.execute(getMoviesQuery)
    moviesList = cursor.fetchall()
        
    print("____________Here are some movies you may like______________________\n")
    if(len(moviesList) >= 10):
        moviesList = random.sample(moviesList,10)
        for i in range(10):
            print(moviesList[i][0])
    else:
        for i in range(len(moviesList)):
            print(moviesList[i][0])
            
        
    #insertNewUser = 'insert into users(userid,age,gender,occupation) values ('
    #insertNewUser = insertNewUser+ "(SELECT MAX(userid) + 1 FROM users)," + str(age) +"," +"'"+ gender + "'"+"," + "'"+occupation +"')"
    #cursor.execute(insertNewUser)
    #conn.commit()
    

#Closing the connection
conn.close()

Welcome to the application!
Are you a new user? ("Yes","No")

No
Please enter your id:

478
Would you like to add some movies ("Yes","No") ?



In [2]:
User = 'nloutas'
Password = 'Wh1teFang_98'
#Servers --> Loutas ---> databases ---> DataMining
#For Server: password = ab159753
#For database = DataMining: username = nloutas
#                           password = Wh1teFang_98 
#Execute SQL queries from the Query Tool in PGAdmin4 or from here